### Import Libraries

In [2]:
import cv2
import warnings
from ultralytics import YOLO, solutions
warnings.filterwarnings('ignore')

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\Zee\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


### Set up the video capture with a local video file

In [4]:
cap = cv2.VideoCapture(r"C:\Users\Zee\Desktop\Human Counting and Ethnicity Detection\video_7.mp4")  # replace with your video file path
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

### Load the YOLO model

In [5]:
model = YOLO("yolov8n.pt")

100%|██████████| 6.25M/6.25M [00:32<00:00, 199kB/s]


### Define initial vertical position for object counting line

In [6]:
initial_position = int(h * 0.76)  # initial position as a percentage of video height


### Function to update region points

In [7]:
def update_region_points(position):
    global region_points
    region_points = [
        (int(w * 0.02), position),
        (int(w * 0.98), position),
        (int(w * 0.98), position - int(h * 0.08)),
        (int(w * 0.02), position - int(h * 0.08))
    ]

In [8]:
# Initialize region points with the initial position
update_region_points(initial_position)

### Video writer

In [9]:
video_writer = cv2.VideoWriter("combined_output.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

### Init Object Counter

In [10]:
counter = solutions.ObjectCounter(
    view_img=False,
    reg_pts=region_points,
    classes_names=model.names,
    draw_tracks=True,
    line_thickness=2,
)

requirements: Ultralytics requirement ['shapely>=2.0.0'] not found, attempting AutoUpdate...
   ---------------------------------------- 1.4/1.4 MB 127.3 kB/s eta 0:00:00

requirements: AutoUpdate success  25.8s, installed 1 package: ['shapely>=2.0.0']
requirements:  Restart runtime or rerun command for updates to take effect

Ultralytics Solutions:  {'region': None, 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None, 'records': 5, 'view_img': False, 'reg_pts': [(12, 273), (627, 273), (627, 245), (12, 245)], 'classes_names': {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 2

100%|██████████| 5.35M/5.35M [00:47<00:00, 117kB/s] 


### Process video frames

In [16]:
while True:
    # Capture a frame from the video
    ret, frame = cap.read()
    if not ret:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    # Perform object tracking
    results = model.track(frame, persist=True, show=False)

    # Count objects
    frame = counter.count(frame)

    # Write the processed frame to the output video
    video_writer.write(frame)

    # Display the frame with object counting
    cv2.imshow('Combined Video', frame)

    # Exit the loop if the user presses the 'Escape' key
    if cv2.waitKey(1) & 0xFF == 27:  # ASCII value of 'Escape' key is 27
        break


0: 384x640 6 persons, 1 cup, 384.5ms
Speed: 14.6ms preprocess, 384.5ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 1 backpack, 1 handbag, 1 suitcase, 220.9ms
Speed: 5.4ms preprocess, 220.9ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 448.6ms
Speed: 11.7ms preprocess, 448.6ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 1 suitcase, 211.2ms
Speed: 3.5ms preprocess, 211.2ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 persons, 1 tv, 196.6ms
Speed: 4.4ms preprocess, 196.6ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 1 suitcase, 198.2ms
Speed: 6.6ms preprocess, 198.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 8 persons, 1 tv, 234.1ms
Speed: 10.3ms preprocess, 234.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 pe

### Release resources

In [17]:
cap.release()
video_writer.release()
cv2.destroyAllWindows()